# Testing
### Eintragung von Partnerschaften nach Geschlecht und Herkunft, seit 2014
Datum: 02.11.2021

**Importiere die notwendigen Packages**

In [1]:
#pip install geopandas fiona requests folium mplleaflet contextily

In [4]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [5]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_partnerschaft_geschlecht_herkunft_seit2014_od3321

(int_dwh)

In [6]:
# Filepath
fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bev_partnerschaft_geschlecht_herkunft_seit2014_od3321\BEV332OD3321.csv"


# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )


In der Folge ein paar erste Tests:

In [7]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [8]:
data2betested.head(3)

,EreignisDatJahr,SexLang,SexCd,HerkunftKomb,HerkunftKombCd,Anzahl
0,2014,männlich,1,CHmitCH,1,20
1,2014,männlich,1,CHmitAus,2,40
2,2014,männlich,1,AusmitAus,3,25


In [9]:
data2betested.dtypes

EreignisDatJahr     int64
SexLang            object
SexCd               int64
HerkunftKomb       object
HerkunftKombCd      int64
Anzahl              int64
dtype: object

In [10]:
data2betested.shape

(42, 6)

Beschreibe einzelne Attribute

In [11]:
data2betested.describe()

,EreignisDatJahr,SexCd,HerkunftKombCd,Anzahl
count,42,42,42,42
mean,2017,2,2,21
std,2,1,1,15
min,2014,1,1,3
25%,2015,1,1,8
50%,2017,2,2,14
75%,2019,2,3,25
max,2020,2,3,66


Wie viele Nullwerte gibt es im Datensatz?

In [12]:
data2betested.isnull().sum()

EreignisDatJahr    0
SexLang            0
SexCd              0
HerkunftKomb       0
HerkunftKombCd     0
Anzahl             0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [13]:

data2betested['StichtagDatJahr'] = data2betested['EreignisDatJahr']

data2betested['StichtagDatJahr']=pd.to_datetime(data2betested['StichtagDatJahr'], format='%Y')

#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested

,EreignisDatJahr,SexLang,SexCd,HerkunftKomb,HerkunftKombCd,Anzahl,StichtagDatJahr
0,2014,männlich,1,CHmitCH,1,20,2014-01-01
1,2014,männlich,1,CHmitAus,2,40,2014-01-01
2,2014,männlich,1,AusmitAus,3,25,2014-01-01
3,2014,weiblich,2,CHmitCH,1,12,2014-01-01
4,2014,weiblich,2,CHmitAus,2,7,2014-01-01
5,2014,weiblich,2,AusmitAus,3,11,2014-01-01
6,2015,männlich,1,CHmitCH,1,25,2015-01-01
7,2015,männlich,1,CHmitAus,2,43,2015-01-01
8,2015,männlich,1,AusmitAus,3,32,2015-01-01
9,2015,weiblich,2,CHmitCH,1,9,2015-01-01


Bereite die Daten für die Grafik unten vor

In [14]:
Jahre = data2betested.groupby("StichtagDatJahr")
len(Jahre)

7

In [15]:
for key, group in Jahre:
    print(key, len(group))

2014-01-01 00:00:00 6
2015-01-01 00:00:00 6
2016-01-01 00:00:00 6
2017-01-01 00:00:00 6
2018-01-01 00:00:00 6
2019-01-01 00:00:00 6
2020-01-01 00:00:00 6


In [16]:
myJahr = "2020-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr

,EreignisDatJahr,SexLang,SexCd,HerkunftKomb,HerkunftKombCd,Anzahl,StichtagDatJahr
36,2020,männlich,1,CHmitCH,1,19,2020-01-01
37,2020,männlich,1,CHmitAus,2,41,2020-01-01
38,2020,männlich,1,AusmitAus,3,23,2020-01-01
39,2020,weiblich,2,CHmitCH,1,13,2020-01-01
40,2020,weiblich,2,CHmitAus,2,8,2020-01-01
41,2020,weiblich,2,AusmitAus,3,6,2020-01-01


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [17]:
#test = data2betested.groupby(['StichtagDatJahr', 'AlterV20Kurz'])[['AnzBestWir']].agg()

# produces Pandas Series
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])['AnzBestWir'].sum()
# Produces Pandas DataFrame
    #data2betested.groupby(['StichtagDatJahr', 'KonOGDSort'])[['AnzBestWir']].sum()
    

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [18]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [21]:
data2betested.head(5)

,EreignisDatJahr,SexLang,SexCd,HerkunftKomb,HerkunftKombCd,Anzahl,StichtagDatJahr
0,2014,männlich,1,CHmitCH,1,20,2014-01-01
1,2014,männlich,1,CHmitAus,2,40,2014-01-01
2,2014,männlich,1,AusmitAus,3,25,2014-01-01
3,2014,weiblich,2,CHmitCH,1,12,2014-01-01
4,2014,weiblich,2,CHmitAus,2,7,2014-01-01


In [26]:
# mySelection = data2betested[(data2betested.HerkunftKombCd == 1) & (data2betested.SexLang == "männlich")]
mySelection = data2betested[(data2betested.HerkunftKombCd != 0) & (data2betested.SexLang == "männlich")]
mySelection2 = data2betested[(data2betested.HerkunftKombCd != 0) & (data2betested.SexLang == "weiblich")]
mySelection3 = data2betested[(data2betested.HerkunftKombCd != 0) & (data2betested.SexLang == "männlich")]
mySelection4 = data2betested[(data2betested.HerkunftKombCd != 0) & (data2betested.SexLang == "weiblich")]

highlight = alt.selection(type='single', on='mouseover',
                          fields=['HerkunftKomb'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='HerkunftKomb'
)

second = alt.Chart(mySelection2).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='HerkunftKomb'
)

third = alt.Chart(mySelection3).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='HerkunftKomb'
)

fourth = alt.Chart(mySelection4).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    color='HerkunftKomb'
)


points = base.mark_circle().encode(
    opacity=alt.value(2)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)

lines4 = fourth.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines3 = third.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines2 = second.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()



points + lines + lines2

alt.LayerChart(...)

In [28]:
alt.Chart(data2betested).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='Anzahl',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='greens')),
    color=alt.Color('HerkunftKomb', scale=alt.Scale(scheme='dark2')),
    tooltip=['StichtagDatJahr','HerkunftKomb', 'SexLang','Anzahl']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)

**Sharepoint als gecheckt markieren!**

Record auf Sharepoint: **[Link](http://kollaboration.intranet.stzh.ch/orga/ssz-produkte/Lists/SASA_Outputs/EditForm.aspx?ID=52&Source=%2Forga%2Fssz%2Dprodukte%2FLists%2FSASA%5FOutputs)**